In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Explicit schema (same as batch)
sales_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("order_id", StringType(), True),
    StructField("country", StringType(), True),
    StructField("product", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("amount", DoubleType(), True),
    StructField("ingest_date", DateType(), True)
])

input_path = "/Volumes/retail_lakehouse/bronze/raw_sales_files"
checkpoint_path = "/Volumes/retail_lakehouse/bronze/checkpoints/autoloader"
schema_path = "/Volumes/retail_lakehouse/bronze/schemas"

bronze_stream_df = (
    spark.readStream
         .format("cloudFiles")
         .option("cloudFiles.format", "csv")
         .option("cloudFiles.schemaLocation", schema_path)
         .option("cloudFiles.inferColumnTypes", "false")
         .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
         .option("header", "true")
         .schema(sales_schema)
         .load(input_path)
         .withColumn("ingestion_ts", current_timestamp())
)

(
    bronze_stream_df.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", checkpoint_path)
        .table("retail_lakehouse.bronze.sales_bronze")
)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-7077577998944220>, line 37
     17 schema_path = "/Volumes/retail_lakehouse/bronze/schemas"
     19 bronze_stream_df = (
     20     spark.readStream
     21          .format("cloudFiles")
   (...)
     29          .withColumn("ingestion_ts", current_timestamp())
     30 )
     32 (
     33     bronze_stream_df.writeStream
     34         .format("delta")
     35         .outputMode("append")
     36         .option("checkpointLocation", checkpoint_path)
---> 37         .table("retail_lakehouse.bronze.sales_bronze")
     38 )

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/streaming/readwriter.py:727, in DataStreamWriter.table(self, tableName)
    725 def table(self, tableName: str) -> "StreamingQuery":
    726     """Alias for the toTable API"""
--> 727     return self.toTable(tableName)

File /d